In [1]:
import pandas as pd

c:\Users\aguer\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Carga de Datos

In [3]:
df = pd.read_csv("total-degree-days.xlsx")

#### Normalizar Datos

In [4]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Value"] = df["Value"].astype(str).str.replace(",", ".").astype(float)
Tbase = 18

#### Extraer día y mes

In [5]:
df["day"] = df["Date"].dt.day
df["month"] = df["Date"].dt.month
df["day_of_year"] = df["Date"].dt.dayofyear

#### Calcular nuevos datos

In [6]:
df_daily = (
    df.groupby(["month", "day"])["Value"]
    .agg(Tmin="min", Tmax="max")
    .reset_index()
)
df_daily["Tavg"] = (df_daily["Tmax"] + df_daily["Tmin"]) / 2


#### HDD

In [7]:
def calc_HDD(row):
    Tmax, Tmin, Tavg = row["Tmax"], row["Tmin"], row["Tavg"]
    if Tmax <= Tbase:
        return Tbase - Tavg
    elif Tavg <= Tbase < Tmax:
        return ((Tbase - Tmin) / 2) - ((Tmax - Tbase) / 4)
    elif Tmin < Tbase < Tavg:
        return (Tbase - Tmin) / 4
    else:
        return 0

df_daily["HDD"] = df_daily.apply(calc_HDD, axis=1)

#### Salida

In [8]:
df_daily["Date"] = df_daily.apply(lambda r: f"{int(r['day']):02d}-{int(r['month']):02d}", axis=1)

df_result = df_daily[["Date", "HDD"]].sort_values(by=["Date"])

df_result.to_excel("HDD_por_dia_promedio.xlsx", index=False)
